# Lilsim Python SDK Test Suite

This notebook comprehensively tests all communication features of the lilsim simulator.

**Prerequisites:**
1. Start the lilsim application: `./build/debug/app/lilsim`
2. The tests will automatically enable/disable ZMQ and switch modes
3. Watch the GUI Status panel to see communication state updates

**What's Being Tested:**
- Connection and state streaming
- Admin commands (pause, run, reset, step)
- Asynchronous control mode
- Synchronous control mode
- Mode switching
- Client disconnection and timeout behavior
- Control period configuration (milliseconds → ticks)
- Marker visualization
- State updates continuity

In [5]:
# Setup and imports
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Add the SDK to path
sys.path.insert(0, '.')

from lilsim import LilsimClient, AdminCommandType, MarkerType, ControlRequest
from lilsim.utils import state_to_dict

print("✓ Imports successful!")

✓ Imports successful!


In [6]:
client = LilsimClient(host="localhost")
client.connect()

INFO:lilsim.client:Connecting to lilsim at localhost...
INFO:lilsim.client:Connected to state stream (port 5556)
INFO:lilsim.client:Connected to admin command endpoint (port 5558)
INFO:lilsim.client:Connected to async control stream (port 5559)
INFO:lilsim.client:Connected to marker stream (port 5560)


In [10]:
import numpy as np
from splinepath import SplinePath

# load midpoints
data = np.genfromtxt('/home/will/code/lilsim/tracks/skidpad.csv', delimiter=',', skip_header=1, dtype=str)
midpoint_rows = data[data[:,0] == 'midpoint']
midpoints = midpoint_rows[:,1:3].astype(float)

spline_path = SplinePath(midpoints)

# publish points sampled from spline path as a line strip
spline_points = spline_path.sample(sample_length=0.5)
n_samples = spline_points.shape[0]
spline_points = np.hstack((spline_points, np.zeros((n_samples, 1))))
client.publish_marker(
    ns="spline", id=3, marker_type=MarkerType.LINE_STRIP,
    points=spline_points, r=0, g=100, b=255, a=200, scale_x=0.1
)


In [8]:
print(n_samples)
print(spline_path.length)

534
266.8705912385541


In [ ]:
current_state = None
def state_callback(state):
    state_dict = state_to_dict(state)
    current_state = state_dict


def pursuit_point(path_points, p_car, lookahead):
    """ Return pure-pursuit given the position of the car.
    
        Input:
        p_car - Car position in global coordinates
        
        Return:
        pursuit point in global coordinates
    """

    close_points = []
    for point in path_points:
        d2 = np.linalg.norm(p_car - point)
        if d2 < lookahead:
            close_points.append(point)
    p_purepursuit = close_points[-1]

    return p_purepursuit

def pure_pursuit_control(dp, theta, wheelbase):
    """ Compute pure-pursuit steer angle.
    
        Input:
        dp - Vector from position of car to pursuit point
        theta - heading of vehicle
        
        Output:
        return steer angle
    """

    l = np.linalg.norm(dp)
    rot_mat = np.array([[np.cos(theta - np.pi/2), -np.sin(theta - np.pi/2)],
                        [np.sin(theta - np.pi/2),  np.cos(theta - np.pi/2)]])
    h_x = np.matmul(rot_mat, (np.array([[1, 0]]).reshape(-1, 1)))
    x = np.dot(dp, h_x)
    delta = np.arctan2(-wheelbase*2*x/(l**2), 1)

    return delta[0]
    
def pp_steer_angle(w, path_points, lookahead, wheelbase):
    """ Compute control action
    
        Input:
        w - current state w = (x, y, theta, v)
        
        Output:
        return delta, the steer angle
    """
    x, y, theta, v = w
    p_car = np.array([x, y])

    p_purepursuit = pursuit_point(path_points, p_car, lookahead)
    dp = p_purepursuit - p_car
    delta = pure_pursuit_control(dp, theta, wheelbase)
    
    return delta

def acceleration_proportional(current_v: float, v_setpoint: float, Kp: float) -> float:
    ax = (v_setpoint - current_v) * Kp
    return ax

def visualize_projection(car_pos: tuple[float, float], spline_pos: tuple[float, float]):
    client.publish_line(
        ns="car_to_spline", id=2, from_pos=car_pos, to_pos=spline_pos, r=255, g=0, b=0, a=200
    )
    client.publish_circle(
        ns="car_to_spline", id=0, pos=car_pos, radius=0.2, r=255, g=0, b=0, a=200
    )
    client.publish_circle(
        ns="car_to_spline", id=1, pos=spline_pos, radius=0.2, r=255, g=0, b=0, a=200
    )

# controller params
L = 2.8
lookahead = 4.0
v_max = 5.0
prev_s = 0.0
Kp = 5.0

client.set_params(wheelbase=L, steering_mode="angle")
client.set_mode(sync=True, control_period_ms=100)
def pure_pursuit_controller(request: ControlRequest) -> tuple[float, float, float]:
    tick = request.header.tick
    sim_time = request.header.sim_time
    w = np.array([request.state.car.x, request.state.car.y, request.state.car.yaw, request.state.car.v])
    # w = np.array([current_state['x'], current_state['y'], current_state['yaw'], current_state['v']])

    s, _ = spline_path.project(w[:1], prev_s)
    prev_s = s

    car_pos = (w[0], w[1])
    spline_projection_point = (spline_path.x(s), spline_path.y(s))
    visualize_projection(car_pos, spline_projection_point)

    v_setpoint = v_max if s < spline_path.length - 5.0 else 0.0

    steer_angle = pp_steer_angle(w, spline_points, lookahead, L)
    ax = acceleration_proportional(w[3], v_setpoint, Kp)

    return (steer_angle, 0.0, ax) # (steer_angle, steer_rate, ax)

client.register_sync_controller(pure_pursuit_controller)

client.subscribe_state(state_callback)

In [ ]:
client.reset()
client.run()
client.start()

In [33]:
# client.stop()

In [31]:
client.set_mode(sync=False)
client.send_control_async(steer_rate=-0.00, ax=-0)

INFO:lilsim.client:Admin command succeeded: Asynchronous mode enabled


In [37]:
client.clear_markers()

In [15]:
print("\nTesting visualization markers...")

# Circle marker
print("  - Publishing red circle at (5, 5)...")
client.publish_marker(
    ns="test", id=1, marker_type=MarkerType.CIRCLE,
    x=5.0, y=5.0, scale_x=2.0, scale_y=2.0,
    r=255, g=0, b=0, a=200
)

# Line strip forming a square
print("  - Publishing green square at (10, 10)...")
square = [(10,10,0), (12,10,0), (12,12,0), (10,12,0), (10,10,0)]
client.publish_marker(
    ns="test", id=2, marker_type=MarkerType.LINE_STRIP,
    points=square, r=0, g=255, b=0, a=255
)

# Circle at origin
print("  - Publishing blue circle at origin...")
client.publish_marker(
    ns="test", id=3, marker_type=MarkerType.CIRCLE,
    x=0.0, y=0.0, scale_x=1.5, scale_y=1.5,
    r=0, g=0, b=255, a=200
)

print("✓ Markers published - check viewport for red circle, green square, and blue circle")


Testing visualization markers...
  - Publishing red circle at (5, 5)...
  - Publishing green square at (10, 10)...
  - Publishing blue circle at origin...
✓ Markers published - check viewport for red circle, green square, and blue circle
